https://www.coches.net/segunda-mano/?MakeId=4&MinPrice=4000&MinYear=2012&MaxYear=2022&MinKms=10000&MaxKms=180000&pg=1

## HEAD

Activar entorno virtual

```python
cd .\Scripts
.\Activate
```

Ejecutar sólo una vez

```
pip install undetected-chromedriver

pip install ipykernel

pip install numpy

pip install pandas

python.exe -m pip install --upgrade pip
```

---
---
---

## FUNCTIONS

In [ ]:
def replaceChars(wrong_char_list, string, replace_by):
	output = string
	for c in wrong_char_list:
		if c in string: output = string.replace(c, replace_by)
	return output

In [ ]:
def loadTextFileFromDisk(PATH):
    with open(PATH, encoding='utf-8', mode = 'r') as f:
        data = f.read()
    f.close()
    return data

def saveToTextFile(PATH, data):
    with open(PATH, encoding='utf-8', mode = 'w') as f:
        f.write(data)
    f.close()

# -----------------------------------------------------

def loadFromDisk(PATH):
    with open(PATH, mode = 'r') as f:
        data = f.read()
    f.close()
    return data

def saveToDisk(PATH, data):
    with open(PATH, mode = 'w') as f:
        f.write(data)
    f.close()

In [ ]:
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
from selenium import webdriver

def get_driver():
  options = webdriver.ChromeOptions()
  options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36')
  options.add_argument("--incognito")
  driver = uc.Chrome(options = options)
  driver.set_page_load_timeout(60)
  driver.set_window_size(1366, 768)
  driver.set_window_position(0, 0)
  driver.delete_all_cookies()
  driver.execute_script("localStorage.clear();")
  driver.execute_script("sessionStorage.clear();")
  driver.execute_script("history.go(0);")
  driver.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {
    'source': 'Object.defineProperty(navigator, "webdriver", {get: () => undefined})'
  })
  return driver

---
---
---

## Obtener todos los enlaces a la web de cada vehículo

Los enlaces se almacenarán en un diccionario cuya estructura es:

>> número de la página en la que se recuperaron

>> enlace a la página de detalle del vehículo

Posteriormente el diccionario se convertirá a `dataframe Pandas`

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

Getting the last page of used cars by make

In [ ]:
LINK = 'https://www.coches.net/segunda-mano/?MakeId=4&MinPrice=4000&MinYear=2012&MaxYear=2022&MinKms=10000&MaxKms=180000&pg=1'
LAST_PAGE = '/html/body/div[1]/div/div[2]/div/main/div[2]/section/div[4]/div/ul/li[7]/a/span'

driver = get_driver()
actions = ActionChains(driver)
driver.get(LINK)

element = driver.find_element(by=By.XPATH, value=LAST_PAGE)

last_page_numeric = int(element.get_attribute('innerHTML'))
print('===============')
print(last_page_numeric)
print('===============')

In [ ]:
# Scrapper
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
from selenium import webdriver

# Utilidades
from collections import defaultdict
from datetime import datetime
import time
import random

# info = defaultdict(list)
info = {'PAGE': list(), 'HREF': list()}

# Constantes
ACCEPT_COOKIES_XPATH = '/html/body/div/div/div[2]/div/div[2]/div/div/div/footer/div/button[2]'
LINK_BASE = 'https://www.coches.net/segunda-mano/?MakeId=4&MinPrice=4000&MinYear=2012&MaxYear=2022&MinKms=10000&MaxKms=180000&pg='
CLASS = 'mt-CardBasic-titleLink'
KEY = '\ue00f' # "PageDown" key

# Setting the number of webs to scrap
if last_page_numeric <= 50:
    PAGE_END = last_page_numeric - 1
else:
    PAGE_END = 50

# Variables
page_current = 1
total_cars = 0

while page_current <= PAGE_END:

    # Con este enlace accedemos a cada página
    # que contiene vehículos de segunda mano
    LINK = LINK_BASE + str(page_current)

    # Se instancia el objeto Selenium/Undetected Chrome-Driver
    driver = get_driver()
    actions = ActionChains(driver)
    
    # Indicamos al Driver qué página debe visitar
    driver.get(LINK)

    try:
        # Movemos el ratón para evitar que nos
        # detecte el agente anti-bots del sitio web
        actions.move_by_offset(random.uniform(-20, 20), random.uniform(-100, 200))
    except:
        pass
    time.sleep(random.uniform(0.5, 1.5))
    try:
        # Cierra la ventana de aceptar "Cookies"
        button = driver.find_element(by = By.XPATH, value = ACCEPT_COOKIES_XPATH)
        try:
            actions.move_to_element(button).perform()
        except:
            pass
        button.click()
    except:
        pass
    time.sleep(random.uniform(0.8, 1.2))
    try:
        # Se recorre toda la página presionando la tecla 'AvanzarPágina' 
        # catorce veces
        for i in range (14):
            ActionChains(driver).send_keys(KEY).perform()
            time.sleep(random.uniform(0.5, 2))
    except:
        pass
    time.sleep(random.uniform(0.3, 1.0))
    try:
        # Almacenamos el contenedor web que contiene
        # toda la información de los coches presentes
        # en la página
        actions.move_by_offset(random.uniform(-20, 20), random.uniform(-100, 200))
        time.sleep(random.uniform(0.5, 2))
        elements = driver.find_elements(by=By.CLASS_NAME, value=CLASS)
    except:
        pass
    time.sleep(random.uniform(0.4, 1.0))
    # Obtener los enlaces a la página de cada vehículo
    for item in elements:
        href = item.get_attribute('href')
        info['PAGE'].append(page_current)
        info['HREF'].append(href)
        total_cars += 1
    # Se actualiza el contador y pasamos a la
    # página siguiente
    page_current += 1
    time.sleep(random.uniform(1.5, 3.5))
    # ---------------------------
    print(f'----------------------------------------')
    print(f'Se ha recorrido la página {page_current-1}')
    print(f'----------------------------------------')
    # ---------------------------


In [ ]:
import pandas as pd

df_info = pd.DataFrame(data = info)

In [ ]:
df_info.head()

,PAGE,HREF
0,1,https://www.coches.net/audi-a5-20-tdi-140kw-19...
1,1,https://www.coches.net/audi-a1-sportback-10-tf...
2,1,https://www.coches.net/audi-a1-12-tfsi-86cv-at...
3,1,https://www.coches.net/audi-q2-design-30-tfsi-...
4,1,https://www.coches.net/audi-a3-16-tdi-s-tronic...


In [ ]:
df_info.tail()

,PAGE,HREF
1166,43,https://www.coches.net/audi-a7-sportback-30-td...
1167,43,https://www.coches.net/audi-a3-16-tdi-sportbac...
1168,43,https://www.coches.net/audi-sq5-30-tdi-326cv-q...
1169,43,https://www.coches.net/audi-rs6-avant-40-tfsi-...
1170,43,https://www.coches.net/audi-a1-attraction-14-t...


In [ ]:
df_info.to_csv('C:\Python\coches_net\data\links.csv', index=False)

---
---
---

## Visitar cada vehículo para obtener toda la información que nos interesa

In [ ]:
from datetime import datetime
from datetime import date

cars_details = {
	'color': [],
	'fuelTypeId': [],
	'fuelType': [],
	'id': [],
	'km': [],
	'make': [],
	'makeId': [],
	'model': [],
	'modelId': [],
	'price': [],
	'province': [],
	'provinceId': [],
	'title': [],
	'transmissionType': [],
	'transmissionTypeId': [],
	'year': [],
	'seller_type': [],
	'bodyType': [],
	'cubicCapacity': [],
	'doors': [],
	'hp': [],
	'url': [],
	'publicationDate': []
}

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
# Scrapper
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
from selenium import webdriver

# Utilidades
from collections import defaultdict
from datetime import date
import time
import random

# info = defaultdict(list)
details = {'PAGE': list(), 'HREF': list()}

# Constantes
CAR_DATA_XPATH = '/html/body/script[3]'
ACCEPT_COOKIES_XPATH = '/html/body/div/div/div[2]/div/div[2]/div/div/div/footer/div/button[2]'
KEY1 = '\ue00f' # "PageDown" key
KEY2 = '\ue00e' # "PageUp" key
TOTAL_CARS = 1

# Variables
index = 0

for i in range(TOTAL_CARS):

    # Con este enlace accedemos a cada página
    # que un vehículo de segunda mano
    # pos = random.uniform(0, len(df_info)-1)
    # LINK = df_info['HREF'].iloc[pos]
    LINK = 'https://www.coches.net/alfa-romeo-stelvio-22-diesel-154kw-210cv-veloce-q4-5p-diesel-2022-en-baleares-54476027-covo.aspx'

    # Se instancia el objeto Selenium/Undetected Chrome-Driver
    driver = get_driver()
    actions = ActionChains(driver)
    
    # Indicamos al Driver qué página debe visitar
    driver.get(LINK)


    time.sleep(random.uniform(30, 35))
    print('AHORA!!')

    try:
        # Movemos el ratón para evitar que nos
        # detecte el agente anti-bots del sitio web
        time.sleep(random.uniform(1.0, 1.5))
        actions.move_by_offset(random.uniform(-50, 10), random.uniform(-150, 60))
    except:
        pass
    time.sleep(random.uniform(0.5, 1.5))
    try:
        # Cerra la ventana de aceptar "Cookies"
        button = driver.find_element(by = By.XPATH, value = ACCEPT_COOKIES_XPATH)
        try:
            actions.move_to_element(button).perform()
        except:
            pass
        button.click()
    except:
        pass
    time.sleep(random.uniform(0.8, 1.2))
    try:
        # Se recorre toda la página presionando la tecla 
        # 'AvanzarPágina' seis veces
        for i in range (6):
            ActionChains(driver).send_keys(KEY1).perform()
            time.sleep(random.uniform(0.5, 2))
        # Movemos el foco de atención a la mitad de 
        # la página (aproximadamente)
        for i in range (int(random.uniform(2, 5))):                
            ActionChains(driver).send_keys(KEY2).perform()
            time.sleep(random.uniform(0.7, 1.3))
    except:
        pass
    time.sleep(random.uniform(0.3, 1.0))
    try:
        # Almacenamos el contenedor web que contiene
        # toda la información de los coches presentes
        # en la página
        actions.move_by_offset(random.uniform(-20, 20), random.uniform(-100, 200))
        time.sleep(random.uniform(0.5, 2))
        elements = driver.find_element(by = By.XPATH, value = CAR_DATA_XPATH)
    except:
        pass
    time.sleep(random.uniform(0.4, 1.0))
    # Obtener los enlaces a la página de cada vehículo
    """
    for item in elements:
        href = item.get_attribute('href')
        info['PAGE'].append(page_current)
        info['HREF'].append(href)
        index += 1
    time.sleep(random.uniform(1.5, 3.5))
    """
    index += 1
    # ---------------------------
    print(f'----------------------------------------')
    print(f'Se ha recorrido el vehículo nº {index}')
    print(f'----------------------------------------')
    # ---------------------------

AHORA!!
----------------------------------------
Se ha recorrido el vehículo nº 1
----------------------------------------


In [ ]:
content = elements.get_attribute('innerHTML')
with open('C:\Python\coches_net\data\data.txt', mode = 'w') as f:
    f.write(content)
f.close()

---
---
---

## Load car details data from disk and put it into dictionary

In [ ]:
PATH = 'C:/Python/coches_net/data/data.txt'
data = loadFromDisk(PATH)

In [ ]:
from datetime import datetime
from datetime import date

cars_details = {
	'color': [],
	'fuelTypeId': [],
	'fuelType': [],
	'id': [],
	'km': [],
	'make': [],
	'makeId': [],
	'model': [],
	'modelId': [],
	'price': [],
	'province': [],
	'provinceId': [],
	'title': [],
	'transmissionType': [],
	'transmissionTypeId': [],
	'year': [],
	'seller_type': [],
	'bodyType': [],
	'cubicCapacity': [],
	'doors': [],
	'hp': [],
	'url': [],
	'publicationDate': []
}

In [ ]:
# Remove \" string from data
data  = data.replace('\\"', '')

wrong_char_list = ['{', '}', ':', '"']

for key in cars_details.keys():
	field = key + ':'
	index = data.find(field)
	# If sub-string was not found return a missing value
	if index < 0:
		target = ''
	else:
		# Put in 'target' the sub-string value found in data according to 'key'
		target = data[index+len(field):index+len(field)+300].split(sep=',')[0]
		# Remove wrong characters returned in 'target' string
		target = replaceChars(wrong_char_list, target, '')
		# Remove trailing and heading wrong characters:
		# Dot, white_space
		target = target.strip('. ')
		# Re-encode string to UTF-8 for language-specific characters
		target = target.encode().decode('unicode_escape')
		# Adding to 'url' field the web site http header
		if key == 'url': target = 'https://www.coches.net' + target
	cars_details[key].append(target)

In [ ]:
df = pd.DataFrame(data = cars_details)
df.to_csv('C:\Python\coches_net\data\car_info.csv', index=False)
df.head()

,color,fuelTypeId,fuelType,id,km,make,makeId,model,modelId,price,...,transmissionType,transmissionTypeId,year,seller_type,bodyType,cubicCapacity,doors,hp,url,publicationDate
0,COMMON.COLOR_BLACK,1,Diésel,54476027,24885,ALFA ROMEO,1,Stelvio,1217,46900,...,COMMON.TRANSMISSION_AUTOMATIC,1,2022,profesional,4x4 SUV,2143,5,210,https://www.coches.net/alfa-romeo-stelvio-22-d...,2023-04-08T102743+0200


---
---
---

## Iterate over links

In [ ]:
import pandas as pd

df = pd.read_csv('C:/Python/coches_net/data/links.csv')
df.head()

,PAGE,HREF
0,1,https://www.coches.net/audi-a5-20-tdi-140kw-19...
1,1,https://www.coches.net/audi-a1-sportback-10-tf...
2,1,https://www.coches.net/audi-a1-12-tfsi-86cv-at...
3,1,https://www.coches.net/audi-q2-design-30-tfsi-...
4,1,https://www.coches.net/audi-a3-16-tdi-s-tronic...


In [ ]:
for link in df['HREF']:
    print(link)

---
---
---

## TEST

In [ ]:
# TEST
wrong_char_list = ['{', '}', ':', '"']

key='fuelType'
field = key + ':'
index = data.find(field)
if index < 0:
	target = ''
else:
	target = data[index+len(field):index+len(field)+300].split(sep=',')[0]
	target = replaceChars(wrong_char_list, target, '')
	target = target.encode().decode('unicode_escape')
	if key == 'url': target = 'https://www.coches.net' + target
print(target)

Diésel


In [ ]:
s = target
s = s.encode().decode('unicode_escape')
print(s)



Diésel


---
---
---

In [ ]:
for index, items in enumerate(info):
    if not items:
        info.pop(index)
# foo = [info.pop(index) for index in range(len(info)) if not info[index]]

In [ ]:
print(len(info))
for item in info:
    print(f'{item}')

In [ ]:
script = driver.find_element(by=By.XPATH, value='/html/body/script[3]')
print(script.get_attribute('innerHTML'))

In [ ]:
'''
    'color': 'COMMON.COLOR_GRAY',
    'fuelTypeId': '1',
    'fuelType': 'Diésel',
    'id': '54087183',
    'km': '105044',
    'make': 'AUDI',
    'makeId': '4',
    'model': 'A6',
    'modelId': '27',
    'price': '29850',
    'province': 'Barcelona',
    'provinceId': '8',
    'title': 'AUDI A6 Black line 2.0 TDI 140 S tro Avant 5p.',
    'transmissionType': 'COMMON.TRANSMISSION_AUTOMATIC',
    'transmissionTypeId': '1',
    'year': '2018',
    'brand': 'AUDI',
    'brand_id': '4',
    'fuel_types': 'Diésel',
    'model_id': '27',
    'seller_type': 'profesional',
    'bodyType': 'BODY_TYPE_FAMILIAR',
    'cubicCapacity': '1968',
    'doors': '5',
    'hp': '190',
    'url': 'https://www.coches.net/audi-a6-black-line-20-tdi-140-s-tro-avant-5p-diesel-2018-en-barcelona-54087183-covo.aspx',
    'datetime': ...,
'''

In [ ]:
LINK = 'https://www.coches.net/segunda-mano/?MakeId=4&MinPrice=4000&MinYear=2012&MaxYear=2022&MinKms=10000&MaxKms=180000&pg=1'
LAST_PAGE = '/html/body/div[1]/div/div[2]/div/main/div[2]/section/div[4]/div/ul/li[7]/a/span'

driver = get_driver()
actions = ActionChains(driver)
driver.get(LINK)

element = driver.find_element(by=By.XPATH, value=LAST_PAGE)

last_page_numeric = int(element.get_attribute('innerHTML'))
print(last_page_numeric)